In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.io import loadmat, savemat
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
p_num=55
a = []

for i in range(10):
    a.append(np.ceil((np.random.permutation(p_num)+1)/(p_num/5.0)).reshape(1,p_num))

a = np.array(a)
len(a)

In [ ]:
normal_Y=np.array([35,45,55,65,83,104,106,108,110,112,126,137,151,154,163,26,53,105]);
normal_O=np.array([1,12,14,20,41,62,77,133,153,15,42,44,64,89,160,161]);
 
mtom_Y=np.array([25,47,50,56,59,63,92,111,51,58,94,102,114,144,122,38,68,91,95,98,138]);
mtom_O=np.array([2,4,96,118,125,142,21,67,101,115,127,136,99,116,130,11,49,82]);
 
severe_Y=np.array([24,28,31,34,36,40,60,76,128,135,57,66,71,79,109,139,152]);
severe_O=np.array([30,43,46,69,70,73,75,86,97,117,129,131,134,156,162,19,23,74,78,84,85]);




control_num=np.array([1,12,14,20,41,62,77,133,153,15,42,44,64,89,160,161,
                     2,4,96,118,125,142,21,67,101,115,127,136,99,116,130,11,49,82,
                     30,43,46,69,70,73,75,86,97,117,129,131,134,156,162,19,23,74,78,84,85]);


Young=np.array([35,45,55,65,83,104,106,108,110,112,126,137,151,154,163,26,53,105,
               25,47,50,56,59,63,92,111,51,58,94,102,114,144,122,38,68,91,95,98,138,
               24,28,31,34,36,40,60,76,128,135,57,66,71,79,109,139,152]);

len(control_num)

In [ ]:
num = 0
for k in range(len(a)):
    b = a[k]
    
    for m in range(1,6):
        index_test = np.where(b == m)
        index_train = np.where(b != m)
        
        test_set = np.empty((48,0))
        train_set = np.empty((48,0))
        test_set_label = np.empty((1,0))
        train_set_label = np.empty((1,0))
        
        test_set_Y = np.empty((48,0))
        test_set_label_Y= np.empty((1,0))
                
        train_set2 = np.empty((48,0))
        test_set_label2 = np.empty((1,0))
        
        test = index_test[1]
        train = index_train[1]
        
        print("{}번째 아우터 루프의 {}번째 테스트 인덱스".format(k, m))
        print(test)
        print("{}번째 아우터 루프의 {}번째 트레인 인덱스 ".format(k, m))
        print(train)
        print("\n")
        
        for n in test:
            test_data = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL.mat".format(control_num[n]))
            test_data = test_data['TOTAL']
            test_set = np.append(test_set, test_data, 1)
            
            test_label = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL_label.mat".format(control_num[n]))
            test_label = test_label['label_TOTAL']
            test_set_label = np.append(test_set_label, test_label, 1)
            
        for m in train:
            train_data = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL.mat".format(control_num[m]))
            train_data = train_data['TOTAL']
            train_set = np.append(train_set, train_data, 1)
            
            train_label = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL_label.mat".format(control_num[m]))
            train_label = train_label['label_TOTAL']
            train_set_label = np.append(train_set_label, train_label, 1)
          
              
        
        train_set2 = np.transpose(train_set)
        train_set_label2 = np.transpose(train_set_label).ravel()
    
        test_set = np.transpose(test_set)
        test_set_label = np.transpose(test_set_label).ravel()
        
        param_grid = {'hidden_layer_sizes':[(10,),(20,),
                                           (10,10,),(10,20,)]}
        clf_grid = GridSearchCV(MLPClassifier(max_iter=200,activation='relu', solver='adam', random_state=1), param_grid) 
        clf_grid.fit(train_set2, train_set_label2)
        hidden_layer_sizes_best = clf_grid.best_params_["hidden_layer_sizes"]
                
        print(hidden_layer_sizes_best)

        model = MLPClassifier(max_iter=200, activation='relu', solver='adam', random_state=1, hidden_layer_sizes=hidden_layer_sizes_best)
        model.fit(train_set2, train_set_label2)
        
        pred = model.predict(test_set)
                
        
        #model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, activation='relu', solver='adam', random_state=1)

        #model.fit(train_set2, train_set_label2)
        #pred = model.predict(test_set)
        
                
        ##### Young test index #########
        c_1 = random.sample(range(len(Young)), len(test))
                
        for n in Young[c_1]:
            test_data_Y = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL.mat".format(n))
            test_data_Y = test_data_Y['TOTAL']
            test_set_Y = np.append(test_set_Y, test_data_Y, 1)
            
            test_label_Y = loadmat("/user/chae2089/CWpaper/FN_REVISION/feature_for_sleep_stage/subject{}_PSD_TOTAL_label.mat".format(n))
            test_label_Y = test_label_Y['label_TOTAL']
            test_set_label_Y = np.append(test_set_label_Y, test_label_Y, 1)
        
        test_set_Y = np.transpose(test_set_Y)
        test_set_label_Y = np.transpose(test_set_label_Y).ravel()
        pred_Y = model.predict(test_set_Y)
        
        
        ######### SAVE O
        result=confusion_matrix(test_set_label,pred)

        filename=("/user/chae2089/CWpaper/FN_REVISION/KFOLD/MLP/old/old_test/result_%d" %(num))
        np.savetxt(filename, result)
        
        report = classification_report(test_set_label, pred)
        report2=report.split()
        with open("/user/chae2089/CWpaper/FN_REVISION/KFOLD/MLP/old/old_test/report_{}.csv".format(num), 'w') as fp:
            for item in report2:
                fp.write("%s\n" % item)
                
                    
        ######### SAVE Y    
        result_Y=confusion_matrix(test_set_label_Y,pred_Y)

        filename=("/user/chae2089/CWpaper/FN_REVISION/KFOLD/MLP/old/young_test/result_%d" %(num))
        np.savetxt(filename, result_Y)
        
        report_Y = classification_report(test_set_label_Y, pred_Y)
        report2_Y=report_Y.split()
        with open("/user/chae2089/CWpaper/FN_REVISION/KFOLD/MLP/old/young_test/report_{}.csv".format(num), 'w') as fp:
            for item in report2_Y:
                fp.write("%s\n" % item)
                 
                
        num = num+1
        

In [ ]:
#c_1 = random.sample(range(len(Young)), len(test))
#Young[c_1]